In [1]:
import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout
from keras.optimizers import Adam
import joblib

USER_ID = 353
PATH_DATA = '../data/user_sequences/'
EXTENSION_TEXT = '.txt'
PATH_USER_DATA = PATH_DATA + f'user_{USER_ID}_sequence' + EXTENSION_TEXT

data = pd.read_csv(PATH_USER_DATA, header=None)
data = data.to_numpy()[:,1:]  # Discard weekday, just for now
print(data[0:3,:])

[[38.7364443 -9.1407651]
 [38.7356124 -9.1396516]
 [38.7368552 -9.1396499]]


In [2]:
print(data.shape)
#X = data[0:-1,:]
#y = data[1:,:]

X = data
y = data


scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
y = scaler.transform(y)

print(X)

joblib.dump(scaler, filename='..\\models\\test\\scaler')

(240, 2)
[[-0.92485022 -0.82391019]
 [-0.97893972 -0.76336385]
 [-0.89813382 -0.76327141]
 [-0.90914157 -0.78931151]
 [-0.52461131 -0.14022313]
 [ 0.77936247  0.66102226]
 [ 0.77453805  0.6602012 ]
 [ 0.9903694   0.69489233]
 [ 0.77453805  0.6602012 ]
 [ 0.97575958  0.7026788 ]
 [-0.93000625 -0.82641687]
 [-0.93089701 -0.82867343]
 [-0.91820526 -0.7596283 ]
 [-0.99462887 -1.00310237]
 [-1.00933622 -0.99874151]
 [ 1.15651306  0.73364525]
 [ 0.98880244  0.70156412]
 [ 1.11891893  1.418882  ]
 [-0.97527914 -0.88011743]
 [-0.9301883  -0.82819493]
 [-0.9332312  -0.63258156]
 [-0.93483718 -0.82353501]
 [-0.92837426 -0.82586225]
 [-0.70017625  0.12717585]
 [ 0.75989572  0.6334488 ]
 [ 0.98977773  0.69723588]
 [ 0.66464253  0.75075157]
 [-0.91299071 -0.79732635]
 [-0.92749    -0.82434519]
 [-0.70212033  0.12618623]
 [ 0.99083104  0.70433723]
 [ 0.77454455  0.6601577 ]
 [ 0.77484364  0.66021208]
 [ 0.77611151  0.66044589]
 [ 0.98603912  0.69457695]
 [-0.55023539  0.28005606]
 [-0.92298417 -0.81

['..\\models\\test\\scaler']

In [ ]:
kf = KFold(n_splits=8)
sequence_length = 3
epochs = 100

best_score = 10
best_model = None
best_test_index = None

for train_index, test_index in kf.split(X):
    X_train = X[train_index, :]
    y_train = y[train_index, :]
    X_test = X[test_index, :]
    y_test = y[test_index, :]

    train_generator = TimeseriesGenerator(X_train, y_train, length=sequence_length, batch_size=1)
    test_generator = TimeseriesGenerator(X_test, y_test, length=3, batch_size=1)

    optimizer = Adam(learning_rate=0.001) #Default = 0.001

    lstm_model = Sequential()
    lstm_model.add(LSTM(50, input_shape=(sequence_length, 2), return_sequences=True))
    #lstm_model.add(Dropout(0.2))
    #lstm_model.add(LSTM(50))
    lstm_model.add(Dense(2))
    lstm_model.compile(optimizer=optimizer, loss='mse')

    lstm_history = lstm_model.fit(train_generator, epochs=epochs, verbose=1, batch_size=1)

    lstm_score = lstm_model.evaluate(test_generator, verbose=0, batch_size=1)

    print(f'LSTM SCORE: {lstm_score}')

    if lstm_score < best_score:
        best_score = lstm_score
        best_model = lstm_model
        best_test_index = test_index


Epoch 1/100
207/207 [==============================] - 2s 3ms/step - loss: 1.0445
Epoch 2/100
207/207 [==============================] - 1s 3ms/step - loss: 1.0324
Epoch 3/100
207/207 [==============================] - 1s 4ms/step - loss: 1.0219
Epoch 4/100
207/207 [==============================] - 1s 3ms/step - loss: 1.0174
Epoch 5/100
207/207 [==============================] - 1s 3ms/step - loss: 1.0161
Epoch 6/100
207/207 [==============================] - 1s 3ms/step - loss: 1.0123
Epoch 7/100
207/207 [==============================] - 0s 2ms/step - loss: 1.0057
Epoch 8/100
207/207 [==============================] - 1s 3ms/step - loss: 1.0016
Epoch 9/100
207/207 [==============================] - 1s 2ms/step - loss: 0.9995
Epoch 10/100
207/207 [==============================] - 0s 2ms/step - loss: 0.9986
Epoch 11/100
207/207 [==============================] - 1s 3ms/step - loss: 0.9868
Epoch 12/100
207/207 [==============================] - 1s 3ms/step - loss: 0.9828
Epoch 13/100


In [ ]:

print(f'Best score: {best_score}')
print(f'Test Index: {best_test_index}')

pred_X = np.expand_dims(X[0:sequence_length, :], 0)
pred = best_model.predict(pred_X)
print(f'Predicted: {pred}')
print(f'True: {X[4,:]}')

best_model.save('..\\models\\test\\model.h5')